## Data Construction and Analysis

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
from fredapi import Fred
from sklearn.metrics import roc_auc_score, roc_curve

MSF_CSV = "MSF_1996_2023.csv"   
fred = Fred(api_key="")
os.makedirs("plots", exist_ok=True)

In [33]:
def to_month_end(s):
    s = pd.to_datetime(s, errors="coerce")
    if isinstance(s, pd.Series):
        return (s + MonthEnd(0)).dt.tz_localize(None)
    elif isinstance(s, pd.DatetimeIndex):
        return (s + MonthEnd(0)).tz_localize(None)
    else:
        raise TypeError("Expected Series or DatetimeIndex")

def map_sic_to_industry(sic):
    try:
        sic = int(sic)
    except (ValueError, TypeError):
        return "Other"
    if   1    <= sic <=  999: return "Agr/Forest/Fish"
    if 1000   <= sic <= 1499: return "Mining"
    if 1500   <= sic <= 1799: return "Construction"
    if 2000   <= sic <= 3999: return "Manufacturing"
    if 4000   <= sic <= 4999: return "Trans/Utilities"
    if 5000   <= sic <= 5199: return "Wholesale"
    if 5200   <= sic <= 5999: return "Retail"
    if 6000   <= sic <= 6799: return "Finance/RE"
    if 7000   <= sic <= 8999: return "Services"
    if 9000   <= sic <= 9999: return "PublicAdmin"
    return "Other"

def create_jump_variable(df, ret_col="RET", id_col="PERMNO", date_col="date", thr=0.10):
    """y_{i,t+1} = 1{|RET_{i,t+1}| > thr} aligned to month t."""
    df = df.sort_values([id_col, date_col]).copy()
    df[ret_col] = pd.to_numeric(df[ret_col], errors="coerce")
    df["RET_t+1"] = df.groupby(id_col)[ret_col].shift(-1)
    df["y_jump"]  = (df["RET_t+1"].abs() > thr).astype(int)
    return df

def winsorize(s, p=0.01):
    lo, hi = s.quantile([p, 1-p])
    return s.clip(lo, hi)

def fred_monthly(code, name):
    s = fred.get_series(code)
    s = s.to_frame(name)
    s.index = pd.to_datetime(s.index)
    s = s.resample("M").last()
    s.index = (s.index + MonthEnd(0)).tz_localize(None)
    return s

def save_roc_plot(y_true, y_proba, title, filename):
    fpr, tpr, _ = roc_curve(y_true, y_proba)
    auc = roc_auc_score(y_true, y_proba)
    ks = max(tpr - fpr)

    fig, ax = plt.subplots(figsize=(6,5))
    ax.plot(fpr, tpr, lw=2, label=f'AUC={auc:.3f}, KS={ks:.3f}')
    ax.plot([0,1],[0,1],"--",lw=1,color="gray")
    ax.set_xlabel("False positive rate")
    ax.set_ylabel("True positive rate")
    ax.set_title(title)
    ax.legend()
    ax.grid(alpha=0.3)
    fig.tight_layout()

    path = os.path.join("plots", filename)
    fig.savefig(path, bbox_inches="tight")
    plt.close(fig)

    return auc, ks

In [34]:
df = pd.read_csv(MSF_CSV)

df["date"]  = to_month_end(df["date"])
df["SICCD"] = pd.to_numeric(df["SICCD"], errors="coerce")
for col in ["RET", "RETX", "vwretd", "VOL", "SHROUT", "PRC"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

df = create_jump_variable(df, ret_col="RET", id_col="PERMNO", date_col="date", thr=0.10)

df["industry"] = df["SICCD"].apply(map_sic_to_industry)
ind_dummies = pd.get_dummies(df["industry"], prefix="ind")
df = pd.concat([df, ind_dummies], axis=1)

df.sort_values(["PERMNO","date"]).head()

/var/folders/rx/4mtzy3y13t9f2njmm29j60pm0000gp/T/ipykernel_12543/3840698337.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(MSF_CSV)


,PERMNO,date,SHRCD,SICCD,TICKER,COMNAM,PERMCO,CUSIP,BIDLO,ASKHI,...,ind_Construction,ind_Finance/RE,ind_Manufacturing,ind_Mining,ind_Other,ind_PublicAdmin,ind_Retail,ind_Services,ind_Trans/Utilities,ind_Wholesale
0,10001,1996-01-31,11,4920.0,EWST,ENERGY WEST INC,7953,36720410,8.750,9.500,...,False,False,False,False,False,False,False,False,True,False
1,10001,1996-02-29,11,4920.0,EWST,ENERGY WEST INC,7953,36720410,8.750,9.500,...,False,False,False,False,False,False,False,False,True,False
2,10001,1996-03-31,11,4920.0,EWST,ENERGY WEST INC,7953,36720410,9.250,9.750,...,False,False,False,False,False,False,False,False,True,False
3,10001,1996-04-30,11,4920.0,EWST,ENERGY WEST INC,7953,36720410,8.625,9.375,...,False,False,False,False,False,False,False,False,True,False
4,10001,1996-05-31,11,4920.0,EWST,ENERGY WEST INC,7953,36720410,8.625,9.000,...,False,False,False,False,False,False,False,False,True,False


In [35]:
unemp_m = fred_monthly("UNRATE",  "unemp")
cpi_m   = fred_monthly("CPIAUCSL","cpi_sa")
indp_m  = fred_monthly("INDPRO",  "ind_prod")
vix_m   = fred_monthly("VIXCLS",  "vix")

macro = unemp_m.join([cpi_m, indp_m, vix_m], how="outer").sort_index()
macro["cpi_yoy"] = 100 * macro["cpi_sa"].pct_change(12)
macro = macro.drop(columns=["cpi_sa"])

jump_share = (df.groupby("date")["y_jump"].mean()
                .rename("jump_frac")
                .to_frame()
                .sort_index())

start_date = jump_share.index.min()
macro = macro.loc[start_date:]

def plot_jump_vs_macro(jump_share, macro_series, macro_label, filename):
    fig, ax1 = plt.subplots(figsize=(12,6))
    ax1.plot(jump_share.index, jump_share["jump_frac"], 
             label="Jump Fraction", linewidth=2, color="tab:blue")
    ax1.set_ylabel("Fraction of Jump Stocks")
    
    ax2 = ax1.twinx()
    ax2.plot(macro_series.index, macro_series, 
             label=macro_label, alpha=0.7, color="tab:red")
    ax2.set_ylabel(macro_label)
    
    l1, lab1 = ax1.get_legend_handles_labels()
    l2, lab2 = ax2.get_legend_handles_labels()
    ax1.legend(l1+l2, lab1+lab2, loc="upper left")
    ax1.grid(alpha=0.3)
    plt.title(f"Jump Fraction vs {macro_label} (monthly)")
    plt.tight_layout()
    fig.savefig(f"plots/{filename}", bbox_inches="tight")
    plt.close(fig)

plot_jump_vs_macro(jump_share, macro["unemp"],   "Unemployment Rate", "jump_fraction_unemp.pdf")
plot_jump_vs_macro(jump_share, macro["cpi_yoy"], "CPI YoY (%)",       "jump_fraction_cpi.pdf")
plot_jump_vs_macro(jump_share, macro["ind_prod"],"Industrial Prod.",  "jump_fraction_indprod.pdf")
plot_jump_vs_macro(jump_share, macro["vix"],     "VIX",               "jump_fraction_vix.pdf")

display(jump_share.join(macro[["unemp","cpi_yoy","ind_prod","vix"]]).corr().round(2))

/var/folders/rx/4mtzy3y13t9f2njmm29j60pm0000gp/T/ipykernel_12543/1552388151.py:7: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  macro["cpi_yoy"] = 100 * macro["cpi_sa"].pct_change(12)


,jump_frac,unemp,cpi_yoy,ind_prod,vix
jump_frac,1.00,-0.10,0.12,-0.25,0.66
unemp,-0.10,1.00,-0.40,-0.25,0.23
cpi_yoy,0.12,-0.40,1.00,0.24,-0.03
ind_prod,-0.25,-0.25,0.24,1.00,-0.31
vix,0.66,0.23,-0.03,-0.31,1.00


In [36]:
panel = df.sort_values(["PERMNO","date"]).copy()

if "vwretd" in panel.columns:
    panel["mkt_adj_ret"] = panel["RET"] - panel["vwretd"]

panel["lag_ret"] = panel.groupby("PERMNO")["RET"].shift(1)
for w in (3, 6, 12):
    panel[f"mom_{w}m"] = (panel.groupby("PERMNO")["RET"]
                            .rolling(w, min_periods=w).mean()
                            .reset_index(level=0, drop=True))
for w in (3, 6, 12):
    panel[f"vol_{w}m"] = (panel.groupby("PERMNO")["RET"]
                            .rolling(w, min_periods=w).std()
                            .reset_index(level=0, drop=True))


panel["turnover"] = panel["VOL"] / (panel["SHROUT"] * 1000.0)
panel["dollar_traded"] = (panel["PRC"].abs() * panel["VOL"]).replace([np.inf,-np.inf], np.nan)
panel["amihud"] = (panel["RET"].abs() / panel["dollar_traded"]).replace([np.inf,-np.inf], np.nan)
panel["abs_prc"] = panel["PRC"].abs()

w = 24
panel["RET"]    = pd.to_numeric(panel["RET"], errors="coerce")
panel["vwretd"] = pd.to_numeric(panel["vwretd"], errors="coerce")
def rolling_beta(g):
    r = g["RET"]
    m = g["vwretd"]
    beta = r.rolling(w, min_periods=w).cov(m) / m.rolling(w, min_periods=w).var()
    return beta.replace([np.inf, -np.inf], np.nan)
panel["beta_24m"] = (
    panel.groupby("PERMNO", group_keys=False)
         .apply(rolling_beta)
)

usrec_m = fred_monthly("USREC", "USREC")
dgs10_m = fred_monthly("DGS10", "DGS10")
tb3ms_m = fred_monthly("TB3MS", "TB3MS")
extra_m = usrec_m.join([dgs10_m, tb3ms_m], how="outer").sort_index()
extra_m["term_spread"] = extra_m["DGS10"] - extra_m["TB3MS"]

macro_m = macro.copy().reset_index().rename(columns={"index": "date"})
extra_m = extra_m.reset_index().rename(columns={"index": "date"})
panel = panel.merge(macro_m, on="date", how="left").merge(extra_m[["date","USREC","term_spread"]], on="date", how="left")

for c in ["turnover","amihud","vix","cpi_yoy"]:
    if c in panel.columns:
        panel[c] = winsorize(panel[c])

panel.head()

,PERMNO,date,SHRCD,SICCD,TICKER,COMNAM,PERMCO,CUSIP,BIDLO,ASKHI,...,dollar_traded,amihud,abs_prc,beta_24m,unemp,ind_prod,vix,cpi_yoy,USREC,term_spread
0,10001,1996-01-31,11,4920.0,EWST,ENERGY WEST INC,7953,36720410,8.750,9.500,...,1533.00000,0.000017,9.12500,NaN,5.6,72.5712,12.53,2.790698,0.0,0.60
1,10001,1996-02-29,11,4920.0,EWST,ENERGY WEST INC,7953,36720410,8.750,9.500,...,4847.00000,0.000003,9.25000,NaN,5.5,73.6207,17.04,2.717031,0.0,1.30
2,10001,1996-03-31,11,4920.0,EWST,ENERGY WEST INC,7953,36720410,9.250,9.750,...,2684.07954,0.000013,9.48438,NaN,5.5,73.5141,18.88,2.843915,0.0,1.38
3,10001,1996-04-30,11,4920.0,EWST,ENERGY WEST INC,7953,36720410,8.625,9.375,...,2881.68750,0.000025,8.81250,NaN,5.6,74.2491,15.83,2.832675,0.0,1.71
4,10001,1996-05-31,11,4920.0,EWST,ENERGY WEST INC,7953,36720410,8.625,9.000,...,888.37500,0.000024,8.62500,NaN,5.6,74.8142,16.07,2.827087,0.0,1.83


* Crisis co-movement: The jump share spikes during known stress episodes (1998, 2008–09, 2020, 2022–23) and rises with broad-market volatility.
* VIX: Moves first or contemporaneously with jump spikes. A sustained rise or break above its long-run median is a useful 1–3 month precursor for higher jump likelihood.
* Industrial production (INDPRO): Negative momentum (flat or falling over 6–12 months) often precedes or coincides with elevated jump shares, providing medium-lead information.
* Unemployment (UNRATE): Lagging indicator, tends to rise after markets have already broken. Good for confirmation, not for advance warning.
* Inflation (CPI YoY): Episode-dependent. In 2021–22 inflation shocks aligned with more jumps via policy and rate volatility, but CPI is not a consistent predictor across cycles.
* Term spread / recession indicator (if included): Inverted term spread tends to appear before stress; recession dummies confirm conditions already underway.

VIX is the strongest short-term precursor, industrial production momentum helps in the medium term, while unemployment and recession flags are mostly confirming signals. CPI provides situational context.


In [37]:
panel2 = panel.dropna(subset=["y_jump"]).copy()

panel2["year"] = panel2["date"].dt.year
def sample_100_firms_per_year(g):
    per = g["PERMNO"].drop_duplicates()
    k = min(100, len(per))
    chosen = per.sample(n=k, random_state=42)
    return g[g["PERMNO"].isin(chosen)]

sampled = (panel2.groupby("year", group_keys=False)
                  .apply(sample_100_firms_per_year)
                  .sort_values(["year","PERMNO","date"])
                  .reset_index(drop=True))

ind_cols = [c for c in sampled.columns if c.startswith("ind_")]
feature_cols = [
    "lag_ret","mkt_adj_ret","mom_3m","mom_6m","mom_12m",
    "vol_3m","vol_6m","vol_12m",
    "turnover","amihud","abs_prc","beta_24m",
    "vix","unemp","ind_prod","cpi_yoy","USREC","term_spread",
] + ind_cols

model_df = sampled.dropna(subset=["y_jump"] + [c for c in feature_cols if c in sampled.columns]).copy()

X = model_df[[c for c in feature_cols if c in model_df.columns]].reset_index(drop=True)
y = model_df["y_jump"].astype(int).reset_index(drop=True)
meta = model_df[["PERMNO","date","year"]].reset_index(drop=True)

print(f"Sampled rows: {len(model_df):,} | Features: {X.shape[1]}")
X.head()

Sampled rows: 24,919 | Features: 30


,lag_ret,mkt_adj_ret,mom_3m,mom_6m,mom_12m,vol_3m,vol_6m,vol_12m,turnover,amihud,...,ind_Finance/RE,ind_Manufacturing,ind_Mining,ind_Other,ind_PublicAdmin,ind_Retail,ind_Services,ind_Trans/Utilities,ind_Wholesale,ind_prod
0,-0.068996,-0.348028,-0.139071,-0.055052,-0.010239,0.167480,0.146254,0.122975,0.004839,3.108322e-09,...,False,False,False,False,False,False,True,False,False,83.784
1,-0.025253,0.090986,0.040068,0.029415,0.028763,0.067096,0.070875,0.067856,0.000980,2.286190e-07,...,False,True,False,False,False,False,False,False,False,83.784
2,-0.183333,0.139877,-0.081214,-0.035923,0.012110,0.207630,0.159682,0.143213,0.004507,3.560667e-08,...,False,True,False,False,False,False,False,False,False,83.784
3,-0.080569,0.178054,-0.007906,-0.023643,-0.004716,0.178884,0.123014,0.117544,0.000176,5.337293e-06,...,False,True,False,False,False,False,False,False,False,83.784
4,-0.272727,-0.330322,-0.251679,-0.115234,-0.077713,0.073636,0.162801,0.173891,0.002780,5.787586e-06,...,False,True,False,False,False,False,False,False,False,83.784


## Stock Jump Prediction

### Model - 1: Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [39]:
def ks_statistic(y_true, y_score):
    fpr, tpr, _ = roc_curve(y_true, y_score)
    return np.max(tpr - fpr)

def make_logit_pipeline():
    return Pipeline([
        ("scaler", StandardScaler(with_mean=True, with_std=True)),
        ("logit", LogisticRegression(
            max_iter=2000, class_weight="balanced", solver="lbfgs", n_jobs=None
        )),
    ])

def fit_eval(train_idx, test_idx, X, y, label=""):
    model = make_logit_pipeline()
    model.fit(X.iloc[train_idx], y.iloc[train_idx])
    proba = model.predict_proba(X.iloc[test_idx])[:, 1]
    auc = roc_auc_score(y.iloc[test_idx], proba)
    ks  = ks_statistic(y.iloc[test_idx], proba)
    return {"label": label, "auc": auc, "ks": ks, "proba": proba, "y_true": y.iloc[test_idx].values}

In [40]:
X = X.astype(float).copy()
y = y.astype(int).copy()
dates = pd.to_datetime(meta["date"])

train_mask = (dates >= "1996-01-01") & (dates <= "2017-12-31")
test_mask  = (dates >= "2018-01-01") & (dates <= "2023-12-31")

res_static = fit_eval(np.where(train_mask)[0], np.where(test_mask)[0], X, y, label="Static 1996-2017 → 2018-2023")
print(f"AUC: {res_static['auc']:.3f} | KS: {res_static['ks']:.3f}")

save_roc_plot(res_static["y_true"], res_static["proba"],
              "Logit ROC — static split",
              "logit_static_roc.pdf")

AUC: 0.693 | KS: 0.290


(0.6933753616445925, 0.29003597753597754)

In [41]:
years = np.arange(2018, 2024)
rows = []

for yr in years:
    train_idx = np.where((dates >= "1996-01-01") & (dates <= f"{yr-1}-12-31"))[0]
    test_idx  = np.where((dates >= f"{yr}-01-01") & (dates <= f"{yr}-12-31"))[0]
    if len(test_idx) == 0 or len(train_idx) == 0:
        continue
    r = fit_eval(train_idx, test_idx, X, y, label=f"{yr-1}→{yr}")
    rows.append({"year": yr, "auc": r["auc"], "ks": r["ks"], "n": len(test_idx)})

roll_table = pd.DataFrame(rows).set_index("year")
display(roll_table)
print(f"Weighted AUC: {np.average(roll_table['auc'], weights=roll_table['n']):.3f} | "
      f"Weighted KS: {np.average(roll_table['ks'],  weights=roll_table['n']):.3f}")

,auc,ks,n
year,,,
2018,0.691131,0.326116,989
2019,0.732563,0.383365,993
2020,0.677599,0.297445,1028
2021,0.695023,0.310881,804
2022,0.693730,0.306114,896
2023,0.685810,0.321205,945


Weighted AUC: 0.696 | Weighted KS: 0.325


In [42]:
lookback_years = 22  

rows = []
for yr in years:
    train_start = pd.Timestamp(f"{yr - lookback_years}-01-01")
    train_end   = pd.Timestamp(f"{yr-1}-12-31")
    test_start  = pd.Timestamp(f"{yr}-01-01")
    test_end    = pd.Timestamp(f"{yr}-12-31")

    train_idx = np.where((dates >= train_start) & (dates <= train_end))[0]
    test_idx  = np.where((dates >= test_start)  & (dates <= test_end))[0]
    if len(test_idx) == 0 or len(train_idx) == 0:
        continue
    r = fit_eval(train_idx, test_idx, X, y, label=f"{train_start.year}-{train_end.year}→{yr}")
    rows.append({"year": yr, "auc": r["auc"], "ks": r["ks"], "n": len(test_idx)})

fixed_table = pd.DataFrame(rows).set_index("year")
display(fixed_table)
print(f"Weighted AUC: {np.average(fixed_table['auc'], weights=fixed_table['n']):.3f} | "
      f"Weighted KS: {np.average(fixed_table['ks'],  weights=fixed_table['n']):.3f}")

,auc,ks,n
year,,,
2018,0.691131,0.326116,989
2019,0.732563,0.383365,993
2020,0.677223,0.295379,1028
2021,0.696146,0.311335,804
2022,0.694169,0.303524,896
2023,0.689158,0.323414,945


Weighted AUC: 0.697 | Weighted KS: 0.324


### Model 2 and 3: LASSO Logistic regression and Ridge Logistic regression

In [43]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, roc_curve

In [44]:
def ks_statistic(y_true, y_score):
    fpr, tpr, _ = roc_curve(y_true, y_score)
    return np.max(tpr - fpr)

def plot_roc(y_true, y_score, title):
    fpr, tpr, _ = roc_curve(y_true, y_score)
    auc = roc_auc_score(y_true, y_score)
    ks  = ks_statistic(y_true, y_score)
    plt.figure(figsize=(6,5))
    plt.plot(fpr, tpr, lw=2, label=f"AUC={auc:.3f}, KS={ks:.3f}")
    plt.plot([0,1],[0,1],"--",color="gray")
    plt.xlabel("False positive rate"); plt.ylabel("True positive rate")
    plt.title(title); plt.legend(); plt.grid(alpha=0.3); plt.tight_layout()
    plt.show()
    return auc, ks

In [45]:
X = X.astype(float).copy()
y = y.astype(int).copy()
dates = pd.to_datetime(meta["date"])

train_mask = (dates >= "1996-01-01") & (dates <= "2017-12-31")
test_mask  = (dates >= "2018-01-01") & (dates <= "2023-12-31")

Xtr, ytr = X.loc[train_mask], y.loc[train_mask]
Xte, yte = X.loc[test_mask],  y.loc[test_mask]

print(Xtr.shape, Xte.shape, ytr.mean().round(4), yte.mean().round(4))

(19264, 30) (5655, 30) 0.3597 0.4046


In [46]:
lasso_cv = LogisticRegressionCV(
    Cs=np.logspace(-3, 2, 20),        
    penalty="l1",
    solver="saga",
    scoring="roc_auc",
    cv=5,
    max_iter=5000,
    class_weight="balanced",
    n_jobs=-1,
    refit=True
)

pipe_lasso = Pipeline([
    ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ("logit",  lasso_cv)
])
pipe_lasso.fit(Xtr, ytr)

best_C = pipe_lasso.named_steps["logit"].C_[0]
print(f"Best C (1/lambda) from CV: {best_C:.4g}")

proba_lasso = pipe_lasso.predict_proba(Xte)[:,1]
auc_lasso, ks_lasso = save_roc_plot(yte, proba_lasso,
                                    "LASSO logistic — out-of-sample ROC",
                                    "lasso_roc.pdf")
print(f"LASSO OOS AUC={auc_lasso:.3f} | KS={ks_lasso:.3f}")

coef = pipe_lasso.named_steps["logit"].coef_.ravel()
nz_mask = (coef != 0)
selected_features = X.columns[nz_mask]
print(f"Post-LASSO selected {nz_mask.sum()} / {X.shape[1]} features")

if nz_mask.sum() == 0:
    topk = np.argsort(np.abs(coef))[-5:]
    selected_features = X.columns[topk]
    print("No strict nonzeros; using top-5 by absolute coefficient instead.")

post_logit = Pipeline([
    ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ("logit",  LogisticRegression(max_iter=5000, class_weight="balanced", solver="lbfgs"))
])
post_logit.fit(Xtr[selected_features], ytr)
proba_post = post_logit.predict_proba(Xte[selected_features])[:,1]
auc_post, ks_post = save_roc_plot(yte, proba_post,
                                  "Post-LASSO logistic — out-of-sample ROC",
                                  "post_lasso_roc.pdf")
print(f"Post-LASSO OOS AUC={auc_post:.3f} | KS={ks_post:.3f}")

coef_tbl = pd.DataFrame({
    "feature": X.columns,
    "coef_lasso": coef,
    "selected_post": X.columns.isin(selected_features)
}).sort_values("coef_lasso", key=lambda s: s.abs(), ascending=False)
display(coef_tbl.head(20))

Best C (1/lambda) from CV: 0.001
LASSO OOS AUC=0.690 | KS=0.281
Post-LASSO selected 4 / 30 features
Post-LASSO OOS AUC=0.683 | KS=0.267


,feature,coef_lasso,selected_post
7,vol_12m,0.224728,True
12,vix,0.094431,True
6,vol_6m,0.074623,True
20,ind_Finance/RE,-0.029081,True
0,lag_ret,0.000000,False
17,term_spread,0.000000,False
28,ind_Wholesale,0.000000,False
27,ind_Trans/Utilities,0.000000,False
26,ind_Services,0.000000,False
25,ind_Retail,0.000000,False


In [47]:
ridge_cv = LogisticRegressionCV(
    Cs=np.logspace(-3, 2, 20),
    penalty="l2",
    solver="lbfgs",               
    scoring="roc_auc",
    cv=5,
    max_iter=5000,
    class_weight="balanced",
    n_jobs=-1,
    refit=True
)

pipe_ridge = Pipeline([
    ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ("logit",  ridge_cv)
])
pipe_ridge.fit(Xtr, ytr)

best_C_r = pipe_ridge.named_steps["logit"].C_[0]
print(f"Best C (1/lambda) from CV [Ridge]: {best_C_r:.4g}")

proba_ridge = pipe_ridge.predict_proba(Xte)[:,1]
auc_ridge, ks_ridge = save_roc_plot(yte, proba_ridge,
                                    "Ridge logistic — out-of-sample ROC",
                                    "ridge_roc.pdf")
print(f"Ridge OOS AUC={auc_ridge:.3f} | KS={ks_ridge:.3f}")

Best C (1/lambda) from CV [Ridge]: 0.001
Ridge OOS AUC=0.690 | KS=0.282


In [48]:
compare = pd.DataFrame({
    "model": ["LASSO", "Post-LASSO", "Ridge"],
    "AUC":   [auc_lasso, auc_post, auc_ridge],
    "KS":    [ks_lasso,  ks_post,  ks_ridge],
    "C":     [best_C,    np.nan,   best_C_r]
})
display(compare)

,model,AUC,KS,C
0,LASSO,0.690195,0.280713,0.001
1,Post-LASSO,0.683218,0.266543,NaN
2,Ridge,0.689923,0.282162,0.001


### Model - 4: K-Nearest Neighbor (KNN)

In [49]:
from sklearn.neighbors import KNeighborsClassifier

In [50]:
dates = pd.to_datetime(meta["date"])

train_mask = (dates >= "1996-01-01") & (dates <= "2012-12-31")  
val_mask   = (dates >= "2013-01-01") & (dates <= "2017-12-31")  
test_mask  = (dates >= "2018-01-01") & (dates <= "2023-12-31")  

Xtr0, ytr0 = X.loc[train_mask], y.loc[train_mask]
Xva,  yva  = X.loc[val_mask],   y.loc[val_mask]
Xte,  yte  = X.loc[test_mask],  y.loc[test_mask]

def misclass_rate(y_true, y_pred):
    return (y_true != y_pred).mean()

def best_threshold_on_validation(y_true_val, proba_val):
    cand = np.unique(np.round(proba_val, 6))
    thr  = cand[np.argmin([(y_true_val != (proba_val >= t)).mean() for t in cand])]
    return float(thr)

In [51]:
def make_knn(k):
    return Pipeline([
        ("scaler", StandardScaler(with_mean=True, with_std=True)),
        ("knn", KNeighborsClassifier(n_neighbors=k, weights="uniform", metric="minkowski", p=2))
    ])

k_grid = [3,5,7,9,11,15,21,31,41,51]

val_results = []
for k in k_grid:
    pipe = make_knn(k)
    pipe.fit(Xtr0, ytr0)
    yva_hat = pipe.predict(Xva)
    val_results.append({"K": k, "val_misclass": misclass_rate(yva, yva_hat)})

val_tbl = pd.DataFrame(val_results).set_index("K").sort_values("val_misclass")
display(val_tbl)
best_k = int(val_tbl.index[0])
print(f"best K on validation: {best_k}")


Xtr, ytr = pd.concat([Xtr0, Xva]), pd.concat([ytr0, yva])
knn_final = make_knn(best_k).fit(Xtr, ytr)
yte_hat_knn = knn_final.predict(Xte)
knn_mis_test = misclass_rate(yte, yte_hat_knn)
print(f"KNN test misclassification (K={best_k}): {knn_mis_test:.4f}")


,val_misclass
K,
41,0.288077
31,0.288689
51,0.290935
21,0.291343
15,0.294406
9,0.299306
7,0.301552
11,0.302164
5,0.303389


best K on validation: 41
KNN test misclassification (K=41): 0.3712


In [52]:
logit = Pipeline([
    ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ("logit",  LogisticRegression(max_iter=5000, class_weight="balanced", solver="lbfgs"))
]).fit(Xtr0, ytr0)

proba_val = logit.predict_proba(Xva)[:,1]
thr_logit = best_threshold_on_validation(yva.values, proba_val)
proba_test = logit.fit(pd.concat([Xtr0, Xva]), pd.concat([ytr0, yva])).predict_proba(Xte)[:,1]
yte_hat_logit = (proba_test >= thr_logit).astype(int)
mis_logit = misclass_rate(yte, yte_hat_logit)
print(f"Logit: val-threshold={thr_logit:.3f} | test misclass={mis_logit:.4f} | test AUC={roc_auc_score(yte, proba_test):.3f}")

lasso_cv = LogisticRegressionCV(
    Cs=np.logspace(-3, 2, 20), penalty="l1", solver="saga",
    scoring="roc_auc", cv=5, max_iter=5000, class_weight="balanced", n_jobs=-1
)
pipe_lasso = Pipeline([("scaler", StandardScaler()), ("logit", lasso_cv)]).fit(Xtr0, ytr0)
proba_val = pipe_lasso.predict_proba(Xva)[:,1]
thr_lasso = best_threshold_on_validation(yva.values, proba_val)
proba_test = pipe_lasso.fit(pd.concat([Xtr0, Xva]), pd.concat([ytr0, yva])).predict_proba(Xte)[:,1]
yte_hat_lasso = (proba_test >= thr_lasso).astype(int)
mis_lasso = misclass_rate(yte, yte_hat_lasso)
print(f"LASSO: val-threshold={thr_lasso:.3f} | test misclass={mis_lasso:.4f} | test AUC={roc_auc_score(yte, proba_test):.3f}")

ridge_cv = LogisticRegressionCV(
    Cs=np.logspace(-3, 2, 20), penalty="l2", solver="lbfgs",
    scoring="roc_auc", cv=5, max_iter=5000, class_weight="balanced", n_jobs=-1
)
pipe_ridge = Pipeline([("scaler", StandardScaler()), ("logit", ridge_cv)]).fit(Xtr0, ytr0)
proba_val = pipe_ridge.predict_proba(Xva)[:,1]
thr_ridge = best_threshold_on_validation(yva.values, proba_val)
proba_test = pipe_ridge.fit(pd.concat([Xtr0, Xva]), pd.concat([ytr0, yva])).predict_proba(Xte)[:,1]
yte_hat_ridge = (proba_test >= thr_ridge).astype(int)
mis_ridge = misclass_rate(yte, yte_hat_ridge)
print(f"Ridge: val-threshold={thr_ridge:.3f} | test misclass={mis_ridge:.4f} | test AUC={roc_auc_score(yte, proba_test):.3f}")

Logit: val-threshold=0.499 | test misclass=0.3560 | test AUC=0.693
LASSO: val-threshold=0.516 | test misclass=0.3443 | test AUC=0.690
Ridge: val-threshold=0.467 | test misclass=0.3706 | test AUC=0.690


In [53]:
compare = pd.DataFrame({
    "model": ["KNN", f"Logit@{thr_logit:.2f}", f"LASSO@{thr_lasso:.2f}", f"Ridge@{thr_ridge:.2f}"],
    "test_misclass": [knn_mis_test, mis_logit, mis_lasso, mis_ridge]
}).sort_values("test_misclass")
display(compare)

,model,test_misclass
2,LASSO@0.52,0.344297
1,Logit@0.50,0.355968
3,Ridge@0.47,0.370645
0,KNN,0.371176


### Models - 5: XGBOOST

In [54]:
from xgboost import XGBClassifier

In [55]:
dates = pd.to_datetime(meta["date"])
train_mask = (dates >= "1996-01-01") & (dates <= "2012-12-31")  
val_mask   = (dates >= "2013-01-01") & (dates <= "2017-12-31")  
test_mask  = (dates >= "2018-01-01") & (dates <= "2023-12-31")  

Xtr0_ = Xtr0.astype(np.float32).values
Xva_  = Xva.astype(np.float32).values
Xte_  = Xte.astype(np.float32).values
ytr0_ = ytr0.values
yva_  = yva.values
yte_  = yte.values

def misclass_rate(y_true, y_pred):
    return (y_true != y_pred).mean()

def best_threshold_on_validation(y_true_val, proba_val):
    cand = np.unique(np.round(proba_val, 6))
    if len(cand) == 0:
        return 0.5
    losses = [(y_true_val != (proba_val >= t)).mean() for t in cand]
    return float(cand[int(np.argmin(losses))])

pos = ytr0.sum()
neg = len(ytr0) - pos
scale_pos_weight = (neg / pos) if pos > 0 else 1.0
scale_pos_weight

1.6148525664361122

In [56]:
rounds_grid = [50, 100, 200, 400, 800]

val_rows = []
thr_by_round = {}

for n in rounds_grid:
    xgb = XGBClassifier(
        n_estimators=n,
        learning_rate=0.05,
        max_depth=3,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        reg_alpha=0.0,
        objective="binary:logistic",
        eval_metric="logloss",
        tree_method="hist",
        random_state=42,
        scale_pos_weight=scale_pos_weight,
        n_jobs=-1
    )
    xgb.fit(Xtr0_, ytr0_)
    p_val = xgb.predict_proba(Xva_)[:,1]
    thr = best_threshold_on_validation(yva_, p_val)
    thr_by_round[n] = thr
    y_val_hat = (p_val >= thr).astype(int)
    val_rows.append({"n_estimators": n, "val_misclass": misclass_rate(yva_, y_val_hat)})

xgb_val_tbl = pd.DataFrame(val_rows).set_index("n_estimators").sort_values("val_misclass")
display(xgb_val_tbl)

best_rounds = int(xgb_val_tbl.index[0])
best_thr_xgb = thr_by_round[best_rounds]
print(f"best rounds={best_rounds}, validation-opt threshold={best_thr_xgb:.3f}")

Xtr_ = pd.concat([Xtr0, Xva]).astype(np.float32).values
ytr_ = pd.concat([ytr0, yva]).values

xgb_final = XGBClassifier(
    n_estimators=best_rounds,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    reg_alpha=0.0,
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method="hist",
    random_state=42,
    scale_pos_weight=scale_pos_weight,
    n_jobs=-1
).fit(Xtr_, ytr_)

p_test = xgb_final.predict_proba(Xte_)[:,1]
y_test_hat = (p_test >= best_thr_xgb).astype(int)
mis_xgb = misclass_rate(yte_, y_test_hat)
auc_xgb = roc_auc_score(yte_, p_test)

auc_xgb, ks_xgb = save_roc_plot(
    yte_, 
    p_test, 
    "XGBoost — out-of-sample ROC", 
    "xgboost_roc.pdf"
)

print(f"XGBoost: rounds={best_rounds}, thr={best_thr_xgb:.3f} "
      f"| test misclass={mis_xgb:.4f} | test AUC={auc_xgb:.3f} | KS={ks_xgb:.3f}")

,val_misclass
n_estimators,
100,0.270927
50,0.271948
200,0.273581
400,0.277869
800,0.281135


best rounds=100, validation-opt threshold=0.584
XGBoost: rounds=100, thr=0.584 | test misclass=0.3330 | test AUC=0.711 | KS=0.326


### Output PDF

In [57]:
from PyPDF2 import PdfMerger
import os

plot_dir = "plots"
output_pdf = "all_plots.pdf"

files = [f for f in os.listdir(plot_dir) if f.lower().endswith(".pdf")]
files = sorted(files)

merger = PdfMerger()
for f in files:
    path = os.path.join(plot_dir, f)
    merger.append(path)

merger.write(output_pdf)
merger.close()

print(f"Merged {len(files)} PDFs into {output_pdf}")

Merged 9 PDFs into all_plots.pdf
